In [250]:
from mip import *
import math
import queue

In [251]:
def ler_arquivo(file: str):
    restricoes = []
    objetivo = []
    n_var = n_res = 0

    with open(file, "r") as arquivo:
        primeira_linha = arquivo.readline()
        info_quant = [int(n) for n in primeira_linha.split()]

        n_var = info_quant[0]
        n_rest = info_quant[1]
        
        objetivo = [int(n) for n in arquivo.readline().split()]
        
        for linha in arquivo:
            restricoes.append([int(n) for n in linha.split()])
    
    return n_var, n_rest, restricoes, objetivo
    

In [252]:
n_var, n_rest, restricoes, objetivo = ler_arquivo("teste1.txt")

print(f'Coeficientes da função objetivo: {objetivo}')
print(f'Coeficiente de restrições: {restricoes}')
print(f'Número de variáveis: {n_var}')
print(f'Número de restricoes: {n_rest}')    

Coeficientes da função objetivo: [9, 7, 10, 7, 9, 6, 8, 4, 9]
Coeficiente de restrições: [[4, 9, 4, 1, 9, 6, 3, 6, 1, 40], [3, 7, 8, 7, 6, 3, 5, 9, 4, 80], [9, 3, 6, 5, 7, 1, 1, 3, 9, 40], [5, 9, 6, 5, 9, 7, 8, 7, 8, 10], [7, 7, 4, 1, 3, 4, 8, 1, 9, 10], [1, 6, 6, 1, 6, 7, 3, 8, 7, 10], [6, 6, 8, 6, 10, 8, 1, 4, 4, 70], [9, 1, 9, 7, 10, 5, 6, 2, 5, 10], [2, 7, 6, 5, 1, 1, 9, 2, 1, 20]]
Número de variáveis: 9
Número de restricoes: 9


In [253]:
def show(m):
    #m.optimize()
    print(type(m.objective_value))
    print(type(m.vars))
    print(f"Solution value  = {m.objective_value:.4f}\n")

    print(f"type: {m.vars}") 

    print("Solution:")
    for v in m.vars:
        print(f"{v.name} = {v.x:.4f}")

In [254]:
def all_binaries(vars):
    return all(n.x == 0 or n.x == 1 for n in vars)

In [255]:
def criar_modelo():
    m = Model(sense=MAXIMIZE, solver_name=CBC)

    # Criando variáveis
    x = [m.add_var(var_type=CONTINUOUS,name=f'x_{i+1}') for i in range(n_var)]

    # Função objetivo:
    m.objective = xsum(x[i] * objetivo[i] for i in range(n_var))

    # Adicionando restricoes:
    for r in range(n_rest):
        m += xsum(x[i] * restricoes[r][i] for i in range(n_var)) <= restricoes[r][-1]

    return m

In [256]:
def criar_novos_nos(m):
    m_0 = m.copy()
    m_1 = m.copy()

    valor = math.inf
    for v in m.vars:
        if (v.x != 1 and v.x != 0 and (abs(v.x - 0.5)) < valor):
            valor = v.x
            var_escolhido = v

    m_0 += var_escolhido == 1
    m_1 += var_escolhido == 0

    return m_0, m_1

In [257]:
nos = queue.Queue()

m = criar_modelo()
nos.put(m)

bestOfAll = m.copy()

dual = math.inf
primal = -math.inf

 Início:
 1. criar_modelo
 2. Enquanto a fila não estiver vazia:
   Obs.: Se for vazia, então novos nós não foram criados. Assim, o algoritmo chegou a uma resposta
 3. Pega o primeiro da fila
 4. Resolve o modelo 
 5. Se todos os números forem inteiros:
   - Verifica se o valor da função atual é menor que o primal. Se for, então substitui. Se não, pode desconsiderar
 6. Se não, então escolhe a variável que está mais próxima de 0.5
 7. Ramifica o modelo por essa variável escolhida e adiciona os dois nós criados na fila 
 Roda o loop novamente  

In [258]:
teste = criar_modelo()
teste.optimize()
show(teste)

<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 13.5714

type: <mip.lists.VarList object at 0x000002D145AFB950>
Solution:
x_1 = 0.0000
x_2 = 0.0000
x_3 = 0.7143
x_4 = 0.0000
x_5 = 0.0000
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.0000
x_9 = 0.7143


In [259]:
# a poda de um nó: ou seja, quando paramos de criar novas nós a partir dele ocorre quando:
# 1. Inviável
# 2. Menor que o primal: avalia se vale a pena continuar, ou seja,
# se o valor atingido for menor que o primal, então poda

while (not(nos.empty())):
    current = nos.get()

    status = current.optimize()

    if (status != OptimizationStatus.OPTIMAL):
        continue

    show(current)

    if (current.objective_value <= primal):
        continue

    if (all_binaries(current.vars) and current.objective_value > primal):
        bestOfAll = current
        primal = current.objective_value
        continue
    elif (current.objective_value < dual):
        dual = current.objective_value

    print(f"dual {dual}")
    print(f"primal= {primal}")
    
    m_0, m_1 = criar_novos_nos(current)
    nos.put(m_0)
    nos.put(m_1)

<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 13.5714

type: <mip.lists.VarList object at 0x000002D145AF9790>
Solution:
x_1 = 0.0000
x_2 = 0.0000
x_3 = 0.7143
x_4 = 0.0000
x_5 = 0.0000
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.0000
x_9 = 0.7143
dual 13.571428571428573
primal= -inf
<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 12.1429

type: <mip.lists.VarList object at 0x000002D145A4BFB0>
Solution:
x_1 = 0.0000
x_2 = 0.0000
x_3 = 0.2143
x_4 = 0.1429
x_5 = 0.0000
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.0000
x_9 = 1.0000
dual 12.142857142857142
primal= -inf
<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 13.4667

type: <mip.lists.VarList object at 0x000002D145A4BE90>
Solution:
x_1 = 0.0000
x_2 = 0.4000
x_3 = 1.0667
x_4 = 0.0000
x_5 = 0.0000
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.0000
x_9 = 0.0000
dual 12.142857142857142
primal= -inf
<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 12.0000

type: <mip.lists.VarList object at 0x000002D145A4B950>
Solu

In [260]:
show(bestOfAll)

<class 'float'>
<class 'mip.lists.VarList'>
Solution value  = 10.0000

type: <mip.lists.VarList object at 0x000002D145A6B6B0>
Solution:
x_1 = 0.0000
x_2 = 0.0000
x_3 = 1.0000
x_4 = 0.0000
x_5 = 0.0000
x_6 = 0.0000
x_7 = 0.0000
x_8 = 0.0000
x_9 = 0.0000
